In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('training_set.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,133,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


601.8363151550293


In [4]:
train.columns

Index(['training_data.innovation_challenge_key', 'training_data.renewed_yorn',
       'training_data.instance_id', 'training_data.contract_line_id',
       'training_data.minor_line_yorn', 'training_data.major_line_instance_id',
       'training_data.installation_date',
       'training_data.product_sales_order_type',
       'training_data.product_purchase_order_type',
       'training_data.instance_status',
       ...
       'training_data.service_distributor_base_theater_name',
       'training_data.service_distributor_base_distributor_normalized_name',
       'training_data.contract_line_net_usd_amount',
       'training_data.product_net_price',
       'training_data.sales_node_renewal_rate',
       'training_data.customer_renewal_rate',
       'training_data.partner_renewal_rate',
       'training_data.product_renewal_rate',
       'training_data.service_sales_node_installed_base_sales_node_renewal_rate',
       'training_data.service_partner_installed_base_partner_renewal_rate'],


In [6]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['training_data.renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

11.454933404922485


In [48]:
train_copy=train
test_copy=test

In [55]:
#restore train and test data using the copy
train=train_copy
test=test_copy

In [7]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('training_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0013887882232666016


In [8]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

15.87989330291748


In [9]:
#remove columns which have any null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

58.19311213493347


In [10]:
len(unclean_columns)

103

In [11]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in test.columns:
    if(test[col].dtype=='object' and len(test[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

14.795108079910278


In [12]:
# 56 messy columns are removed
len(messy_columns)

56

In [13]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

6.850026369094849


In [14]:
train.shape

(6010041, 77)

In [15]:
#Remove the primary key column for data fitting
t1=time.time()
train_X = train.drop('innovation_challenge_key',axis=1)
test_X = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

3.563671350479126


In [16]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train_X.columns:
    if (train_X[col].dtype !='object'):
        if(len(train_X[col].unique())<10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.18131399154663086


In [17]:
#handle catcols
t1=time.time()
for col in catcols:
    train_X[col]=train_X[col].astype('category')
    test_X[col]=test_X[col].astype('category')
t2=time.time()
print(t2-t1)

0.6053271293640137


In [18]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train_X[scalcols])
train_X[scalcols] = scaler.transform(train_X[scalcols])
scaler = scaler.fit(test_X[scalcols])
test_X[scalcols] = scaler.transform(test_X[scalcols])
t2=time.time()
print(t2-t1)

3.7523045539855957


In [19]:
train_X[scalcols].describe()

,contract_line_net_usd_amount,product_net_price
count,6.010041e+06,6.010041e+06
mean,-3.177913e-17,5.735377e-17
std,1.000000e+00,1.000000e+00
min,-1.164629e-01,-2.451377e-01
25%,-1.146668e-01,-2.451377e-01
50%,-1.146668e-01,-2.451377e-01
75%,-1.146668e-01,-1.634183e-01
max,2.899812e+02,7.503942e+01


In [20]:
train_X[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,6010041,6010041,6010041,6010041
unique,3,4,6,6
top,10002,-1,6,6
freq,5316273,5985929,5248934,5945627


In [21]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train_X)
dataset = pd.concat(objs=[train_X, test_X], axis=0)
dataset = pd.get_dummies(dataset)
train_X = dataset[:train_objs_num]
test_X = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

54.00905227661133


In [22]:
train_X.columns

Index(['service_sales_node_base_sales_hierarchy_level',
       'contract_line_net_usd_amount', 'product_net_price',
       'minor_line_yorn_N', 'minor_line_yorn_Y',
       'instance_status_Latest-INSTALLED', 'product_transaction_type_-9999',
       'product_transaction_type_10000', 'product_transaction_type_10002',
       'installed_product_type_NONSERIALIZED',
       ...
       'service_distributor_base_distributor_name_B8F4F748A1FF7CFA5B2F2DD4847809F600E8FC7A3937A7C32BFBEDBCC7358FE0',
       'service_distributor_base_distributor_name_C321254900CAC4B6824721B6611C15CD8A9A824C66B6C1CA7FC9BDE39ABC5B18',
       'service_distributor_base_distributor_name_C868A0ED0FB7D710C0235B586BDE12FC90AE26BA384782868BB4EE0A6A2FC9B3',
       'service_distributor_base_distributor_name_D496D3E841D7F257C8B8314A580E5728BC034FD09FD4360908B2103860CC728F',
       'service_distributor_base_distributor_name_E15FFA65FCD7A17CF3F1FE025C496E1B5C5D6A5126918C5FFEFFC411240000F5',
       'service_distributor_base_distrib

In [23]:
#print the new columns we got
train_X.shape

(6010041, 295)

In [24]:
test_X.shape

(1888195, 295)

In [25]:
#Replace all NaN left with 0
t1=time.time()
train_X = train_X.fillna(0)
test_X = test_X.fillna(0)
t2=time.time()
print(t2-t1)

0.7320420742034912


In [26]:
#check data quality now
train_X.isnull().any().any()

False

In [27]:
#check data quality now
test_X.isnull().any().any()

False

In [28]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_X,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

15.782866477966309


In [29]:
#Dimensions of training data
X_train.shape

(4026727, 295)

In [64]:
#Lets start fitting different models on the data
#1. start with simple Bayes model
t1=time.time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gaussian_model = gnb.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

In [65]:
# Lets see the prediction on our validation data
t1=time.time()
pred = gaussian_model.predict_proba(X_valid)
t2=time.time()
print(t2-t1)

In [66]:
pred[1,]

array([5.32419453e-22, 1.00000000e+00])

In [67]:
train_y.head()

0    Y
1    Y
2    Y
3    N
4    N
Name: renewed_yorn, dtype: object

In [68]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

9.149762309991958

In [38]:
#find predicted value on test set
pred = gaussian_model.predict_proba(test_X)

In [39]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]

In [40]:
os.chdir('/users/hdpsndbx125')

In [41]:
#very important to sort the values else won't be taken at submission
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')

In [42]:
#persist to a file
submit.to_csv('NaiveBayesGaussian_EntireTraining.csv',index=False)

In [43]:
submit.shape

(1888195, 2)

In [46]:
#fitting a multinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
#find predicted value on test set
pred = clf.predict_proba(test_X)

In [47]:
from sklearn.naive_bayes import BernoulliNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train,y_train)
pred = mnb.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
from sklearn.linear_model import LogisticRegression as lr
t1=time.time()
lr1=lr(max_iter=100)
lr1.fit(X_train,y_train)
pred = lr1.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
print(t2-t1)

In [50]:
#find predicted value on test set
pred = lr1.predict_proba(test_X)

In [51]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('LR_100iter.csv',index=False)

In [2]:
submit.shape

NameError: name 'submit' is not defined

In [56]:
from sklearn.linear_model import SGDClassifier
t1=time.time()
sgd = SGDClassifier(loss="log", penalty="l2",max_iter=1000)
sgd.fit(X_train,y_train)
pred = sgd.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
print(t2-t1)

/apps/dslab/anaconda/python3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


8.873491261204586

In [28]:
#trying Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
t1=time.time()
rfc = RandomForestClassifier(max_depth=4, random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
print(t2-t1)

72.06181812286377


In [29]:
pred = rfc.predict_proba(test_X)

In [30]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('RFC_depth4.csv',index=False)

In [ ]:
from sklearn import svm
import time
t1=time.time()
svc= svm.SVC(kernel='rbf',probability=True)
svc.fit(X_train, y_train)
t2=time.time()
pred = svc.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

In [1]:
log_loss(y_valid,pred[:,1])

NameError: name 'log_loss' is not defined

In [31]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier()
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(train_X, train_y)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()

/apps/dslab/anaconda/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [32]:
t2-t1

591.9079170227051

In [33]:
pred=xgb1.predict_proba(X_valid)

In [34]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

0.5472705340877987

In [35]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test_X)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-entiretrainscl.csv',index=False)

In [30]:
#Trying MLPerceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [33]:
t1=time.time()
mlp.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

MemoryError: 

In [43]:
from sklearn.metrics import log_loss
pred = mlp.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.4238643172502673

In [33]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']

In [35]:
pred=mlp.predict_proba(test_X)

In [37]:
len(pred)

1888195

In [38]:
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('MLP_20_20_20_scl.csv',index=False)